In [19]:
# Step 1: Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, when, count

In [2]:
# Step 2: Create a SparkSession
spark = SparkSession.builder \
    .appName("LaptopStoreAnalysis") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/02/19 22:59:09 WARN Utils: Your hostname, LPSS046 resolves to a loopback address: 127.0.1.1; using 172.18.179.157 instead (on interface eth0)
24/02/19 22:59:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/19 22:59:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
# Step 3: Read the CSV file into a Spark DataFrame
file_path = "data/laptop_pricing_dataset.csv"
laptop_df = spark.read.csv(file_path, header=True, inferSchema=True)

In [14]:
# Step 4: Perform data transformations and calculations for each business question
# a. Average RAM GB per manufacturer
ram_avg_per_manufacturer = laptop_df.groupBy("Manufacturer").agg(avg("RAM_GB").alias("avg_RAM_GB")).orderBy("avg_RAM_GB", ascending=False)
ram_avg_per_manufacturer.show()

+------------+-----------------+
|Manufacturer|       avg_RAM_GB|
+------------+-----------------+
|       Razer|             16.0|
|     Samsung|             11.2|
|        Dell|8.225352112676056|
|      Huawei|              8.0|
|      Xiaomi|              8.0|
|         MSI|              8.0|
|     Toshiba|              8.0|
|        Asus|7.888888888888889|
|      Lenovo|7.576923076923077|
|        Acer|7.368421052631579|
|          HP|7.346938775510204|
+------------+-----------------+



In [16]:
# b. Average screen size per Screen Type
screen_size_avg_per_type = laptop_df.groupBy("Screen").agg(avg("Screen_Size_cm").alias("avg_screen_size")).orderBy("avg_screen_size", ascending=False)
screen_size_avg_per_type.show()

+---------+------------------+
|   Screen|   avg_screen_size|
+---------+------------------+
|  Full HD|37.968955414012704|
|IPS Panel| 35.84368831168828|
+---------+------------------+



In [23]:
# c. Profiling by manufacturer and screen type
profiling_results = laptop_df.groupBy("Manufacturer") \
    .agg(
        count("Manufacturer").alias("Total_Products"),
        count(when(laptop_df["Screen"] == "IPS Panel", 1)).alias("Total_Products_With_IPS_Panel"),
        count(when(laptop_df["Screen"] == "Full HD", 1)).alias("Total_Products_With_Full_HD")
    )
profiling_results.show()

+------------+--------------+-----------------------------+---------------------------+
|Manufacturer|Total_Products|Total_Products_With_IPS_Panel|Total_Products_With_Full_HD|
+------------+--------------+-----------------------------+---------------------------+
|       Razer|             1|                            0|                          1|
|      Huawei|             1|                            1|                          0|
|      Xiaomi|             1|                            1|                          0|
|          HP|            49|                            8|                         41|
|        Dell|            71|                            9|                         62|
|        Acer|            19|                           14|                          5|
|        Asus|            18|                            2|                         16|
|      Lenovo|            52|                           32|                         20|
|     Samsung|             5|   

In [ ]:
# Step 7: Stop the SparkSession
spark.stop()